In [3]:
from PIL import Image

In [54]:
import boto3

In [22]:
im = Image.open('test.jpg')

In [29]:
im.size

(2469, 13200)

In [68]:
import os
import io

def crop(infile,height,width):
    im = Image.open(infile)
    imgwidth, imgheight = im.size
    for i in range(imgheight//int(height/2)):
        box = (0,i*(height/2), width, i*(height/2)+ height)
        yield im.crop(box)

if __name__=='__main__':
    
    
    s3 = boto3.client('s3')
    
    infile= '3_BSL (3)_mark.jpg'
    height= 132
    width= 2469
    start_num= 0
    for k,piece in enumerate(crop(infile,height,width),start_num):
        img=Image.new('L', (width,height), 255)
        img.paste(piece)
        in_mem_file = io.BytesIO()
        path=os.path.join("{}-{}.png".format(infile.split('.')[0], k))
        #img.save(path)
        img.save(in_mem_file, format='png')
        in_mem_file.seek(0)
        
        s3.upload_fileobj(
        in_mem_file,
        'hankook-tire',
        '{}/{}'.format(infile.split('.')[0],path))